In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
sns.set()
import mpu

In [4]:
# !pip install git+https://github.com/MartinThoma/mpu.git

  Cloning https://github.com/MartinThoma/mpu.git to c:\users\chidorialex\appdata\local\temp\pip-req-build-kehwemyf
  Created wheel for mpu: filename=mpu-0.23.1-py3-none-any.whl size=72529 sha256=067b21d2e082f5c98b08c282386ffcfe0f949c4c4e3a75c9c61c2e3f70a72fda
  Stored in directory: C:\Users\chidorialex\AppData\Local\Temp\pip-ephem-wheel-cache-gm0s5bgk\wheels\68\41\c3\dd0d6f86ef228f25a7b847362cd6d386755fb223828c14915f
Successfully built mpu


  Running command git clone -q https://github.com/MartinThoma/mpu.git 'C:\Users\chidorialex\AppData\Local\Temp\pip-req-build-kehwemyf'


In [2]:
bus = pd.read_csv('bus_stops_clean.csv')
bus.head()

,BusStopCode,RoadName,Description,Latitude,Longitude,svc,len
0,1012,Victoria St,Hotel Grand Pacific,1.296848,103.852536,"63,12e,80,2,51,61,7,32,175,2A,12,33",12.0
1,1013,Victoria St,St. Joseph's Ch,1.297710,103.853225,"197,130,851e,960,960e,133,851,145",8.0
2,1019,Victoria St,Bras Basah Cplx,1.296990,103.853022,"130,12e,2,960,960e,133,12,33",8.0
3,1029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414,"63,197,80,851e,51,61,7,32,7A,175,145,851,63A",13.0
4,1039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491,"63,197,80,851e,51,61,7,32,7A,175,145,851,63A",13.0


In [7]:
mrt = pd.read_csv('mrt_static_clean.csv')
mrt['svc'] = np.nan
mrt['bus count'] = np.nan
mrt.head()

,postal,x,y,lat,long,name,subzone,area,region,code,...,NE,NS,DT,EW,CG,BP,TE,CE,svc,bus count
0,534803,34128.79235,35258.74882,1.335142,103.888389,Tai Seng,Tai Seng,Hougang,North-East,CC11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,539788,33214.93397,36072.52553,1.342501,103.880178,Bartley,Bidadari,Toa Payoh,Central,CC12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,556082,32390.68525,36971.75455,1.350634,103.872772,Serangoon,Serangoon Central,Serangoon,North-East,CC13/NE12,...,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,554533,31431.40547,37079.94437,1.351612,103.864152,Lorong Chuan,Lorong Chuan,Serangoon,North-East,CC14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,579842,29760.77555,37047.15733,1.351316,103.849140,Bishan,Bishan East,Bishan,Central,CC15/NS17,...,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
src = (1.344621, 103.720833)
dest = (1.346341, 103.721363)
mpu.haversine_distance(src, dest)

0.20012446127828387

In [59]:
radius = 0.25
for mrt_row in range(len(mrt)):
    mrt_lat = mrt.loc[mrt_row,'lat']
    mrt_long = mrt.loc[mrt_row,'long']
    svc = ""
    for bus_row in range(len(bus)):
        bus_lat = bus.loc[bus_row,'Latitude']
        bus_long = bus.loc[bus_row,'Longitude']
        dist = mpu.haversine_distance((mrt_lat, mrt_long), (bus_lat, bus_long))
        if dist <= radius:
            svc = svc + bus.loc[bus_row,'svc'] + ','
    svc_list = svc[:-1].replace(" ", "").split(',')
    svc_set_list = list(set(svc_list))
    svcs = ""
    for stop in svc_set_list:
        svcs = svcs + stop + ','
        
    mrt.loc[mrt_row, 'svc'] = svcs[:-1]
    if svc_set_list == ['']:
        print(mrt.loc[mrt_row, 'name'])
        mrt.loc[mrt_row, 'bus count'] = 0
    else:
        mrt.loc[mrt_row, 'bus count'] = len(svc_set_list)

Bukit Brown
Punggol Coast
Mount Pleasant
Marina South
Bayshore


In [60]:
mrt.head()

,postal,x,y,lat,long,name,subzone,area,region,code,...,NE,NS,DT,EW,CG,BP,TE,CE,svc,bus count
0,534803,34128.79235,35258.74882,1.335142,103.888389,Tai Seng,Tai Seng,Hougang,North-East,CC11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"24,70,151,80A,76,58A,43,158,62A,90A,62,90,22,9...",20.0
1,539788,33214.93397,36072.52553,1.342501,103.880178,Bartley,Bidadari,Toa Payoh,Central,CC12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"93,158,129,28",4.0
2,556082,32390.68525,36971.75455,1.350634,103.872772,Serangoon,Serangoon Central,Serangoon,North-East,CC13/NE12,...,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"NR6,135,53,109A,147,506,101,147A,107M,70,116A,...",38.0
3,554533,31431.40547,37079.94437,1.351612,103.864152,Lorong Chuan,Lorong Chuan,Serangoon,North-East,CC14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"105,58",2.0
4,579842,29760.77555,37047.15733,1.351316,103.849140,Bishan,Bishan East,Bishan,Central,CC15/NS17,...,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,"53,88,13A,410W,50,56,71,410,59,55B,410G,13,52,...",19.0


In [61]:
#mrt.to_csv('mrt_with_bus_clean.csv', index=False)

In [3]:
data = pd.read_csv('mrt_with_bus_clean.csv')
data.head()

,postal,x,y,lat,long,name,subzone,area,region,code,...,NE,NS,DT,EW,CG,BP,TE,CE,svc,bus count
0,534803,34128.79235,35258.74882,1.335142,103.888389,Tai Seng,Tai Seng,Hougang,North-East,CC11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"24,70,151,80A,76,58A,43,158,62A,90A,62,90,22,9...",20.0
1,539788,33214.93397,36072.52553,1.342501,103.880178,Bartley,Bidadari,Toa Payoh,Central,CC12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"93,158,129,28",4.0
2,556082,32390.68525,36971.75455,1.350634,103.872772,Serangoon,Serangoon Central,Serangoon,North-East,CC13/NE12,...,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"NR6,135,53,109A,147,506,101,147A,107M,70,116A,...",38.0
3,554533,31431.40547,37079.94437,1.351612,103.864152,Lorong Chuan,Lorong Chuan,Serangoon,North-East,CC14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"105,58",2.0
4,579842,29760.77555,37047.15733,1.351316,103.849140,Bishan,Bishan East,Bishan,Central,CC15/NS17,...,NaN,17.0,NaN,NaN,NaN,NaN,NaN,NaN,"53,88,13A,410W,50,56,71,410,59,55B,410G,13,52,...",19.0


In [26]:
pop = pd.read_csv('pop_area_subzone.csv', thousands=',', na_values='...')
pop.columns = ['name', 'status', 2000, 2010, 2015, 'pop']
pop2020 = pop[['name', 'status', 'pop']]
pop2020_sub = pop2020[pop2020.status=='Subzone']
pop2020_sub

,name,status,pop
1,Ang Mo Kio Town Centre,Subzone,4810.0
2,Cheng San,Subzone,28070.0
3,Chong Boon,Subzone,26500.0
4,Kebun Bahru,Subzone,22620.0
5,Sembawang Hills,Subzone,6850.0
...,...,...,...
382,Springleaf,Subzone,4370.0
383,Yishun Central,Subzone,2620.0
384,Yishun East,Subzone,60670.0
385,Yishun South,Subzone,42240.0


In [27]:
pop2020_sub.dtypes

name       object
status     object
pop       float64
dtype: object

In [28]:
# pop2020_sub.to_csv('pop2020_zone_clean.csv', index=False)

In [35]:
pop2020_sub = pd.read_csv('pop2020_zone_clean.csv')

In [36]:
all_subzones = pop2020_sub.name.unique()
for subzone in data.subzone.unique():
    if subzone in all_subzones:
        print(f'{subzone} \t\t found')
    else:
        print(f'{subzone} \t\t not found')

Tai Seng 		 found
Bidadari 		 found
Serangoon Central 		 found
Lorong Chuan 		 found
Bishan East 		 found
Upper Thomson 		 found
Mount Pleasant 		 found
Tyersall 		 found
Bras Basah 		 found
Farrer Court 		 found
Leedon Park 		 found
One North 		 found
National University Of S'Pore 		 found
Pasir Panjang 1 		 found
Port 		 found
Maritime Square 		 found
City Hall 		 found
City Terminals 		 found
Everton Park 		 found
Central Subzone 		 found
Marina Centre 		 found
Nicoll 		 found
Tanjong Rhu 		 found
Aljunied 		 found
Geylang East 		 found
Changi Airport 		 found
Gombak 		 found
Nassim 		 found
Bencoolen 		 found
Bayfront Subzone 		 found
Chinatown 		 found
China Square 		 found
Jelebu 		 found
Clarke Quay 		 found
Sungei Road 		 found
Kallang Bahru 		 found
Geylang Bahru 		 found
Macpherson 		 found
Kampong Ubi 		 found
Kaki Bukit 		 found
Nature Reserve 		 found
Bedok Reservoir 		 found
Tampines West 		 found
Tampines East 		 found
Simei 		 found
Xilin 		 found
Anak Bukit 		 found
Ho

In [37]:
data['pop'] = np.nan
for data_row in range(len(data)):
    data_sz = data.loc[data_row, 'subzone']
    for pop_row in range(len(pop2020_sub)):
        if data_sz == pop2020_sub.loc[pop_row, 'name']:
            pop = pop2020_sub.loc[pop_row, 'pop']
            data.loc[data_row, 'pop'] = pop

In [38]:
data.head()

,postal,x,y,lat,long,name,subzone,area,region,code,...,NS,DT,EW,CG,BP,TE,CE,svc,bus count,pop
0,534803,34128.79235,35258.74882,1.335142,103.888389,Tai Seng,Tai Seng,Hougang,North-East,CC11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"24,70,151,80A,76,58A,43,158,62A,90A,62,90,22,9...",20.0,13830.0
1,539788,33214.93397,36072.52553,1.342501,103.880178,Bartley,Bidadari,Toa Payoh,Central,CC12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"93,158,129,28",4.0,NaN
2,556082,32390.68525,36971.75455,1.350634,103.872772,Serangoon,Serangoon Central,Serangoon,North-East,CC13/NE12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"NR6,135,53,109A,147,506,101,147A,107M,70,116A,...",38.0,24760.0
3,554533,31431.40547,37079.94437,1.351612,103.864152,Lorong Chuan,Lorong Chuan,Serangoon,North-East,CC14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"105,58",2.0,8490.0
4,579842,29760.77555,37047.15733,1.351316,103.849140,Bishan,Bishan East,Bishan,Central,CC15/NS17,...,17.0,NaN,NaN,NaN,NaN,NaN,NaN,"53,88,13A,410W,50,56,71,410,59,55B,410G,13,52,...",19.0,28310.0


In [39]:
data.to_csv('mrt_with_bus_and_pop_clean.csv', index=False)